In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import data_analysis

## Settings

In [ ]:
DATA_FILE = "input_data.csv"
PARAMETER_COLUMNS_SETS = [
    [
        "G_VH_dB",
        "G_VV_dB",
        "projectedLocalIncidenceAngle",
        "assumed_clay_content",
    ],
    [
        "C11_ILSF9_BI1000_dB",
        "C22_ILSF9_BI1000_dB",
        "Surface_r_ILSF9_BI1000",
        "Volume_g_ILSF9_BI1000",
        "Ratio_b_ILSF9_BI1000",
        "projectedLocalIncidenceAngle",
        "assumed_clay_content",
    ],
]
PARAMETERS_SET = 0 # set of parameters from the list above, counting from 0 for the first one
SELECTED_PARAMETERS = PARAMETER_COLUMNS_SETS[PARAMETERS_SET]
TARGET = "soil_moisture"
SPLIT_RANDOM_STATE = None # integer or None for random split
TUNE_HYPERPARAMETERS = False # True or False to skip
# Dictionary with parameters to tune
# (will be used only if TUNE_HYPERPARAMETERS = True)
HYPERPARAMETERS_GRID = {
    "n_estimators": [1000, 1400], # integer
    "max_depth": [16, 14, 12], # integer, None
    "min_samples_split": [2, 3, 5], # integer
    "min_samples_leaf": [1, 2, 4, 6], # integer
    "max_features": ["sqrt"], # "sqrt", "log2"
    "bootstrap": [True], # True, False
}
EXPORT_MODEL = True # True or False to skip

In [ ]:
if not SPLIT_RANDOM_STATE:
    from random import randint

    SPLIT_RANDOM_STATE = randint(1, 1000)
    print(f"Random split state: {SPLIT_RANDOM_STATE}")

## Set default font for graphs

In [ ]:
mpl.rcParams["font.family"] = "Palatino Linotype"

## Read data

In [ ]:
df = pd.read_csv(DATA_FILE, sep=",", engine="python")
df = df[[TARGET] + SELECTED_PARAMETERS]
df.head()

## Exploratory data analysis

In [ ]:
data_analysis.correlation_matrix_heatmap(df, output_file=f"rf_full_parameters_set_{PARAMETERS_SET}")

## Prepare data for training

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
bins = [0, 10, 20, 30, 40, 50, 60]
labels = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60"]
df["moisture_bin"] = pd.cut(df[TARGET], bins=bins, labels=labels)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SPLIT_RANDOM_STATE)

for train_index, validation_index in split.split(df, df["moisture_bin"]):
    df_stratified_training = df.loc[train_index]
    df_stratified_validation = df.loc[validation_index]

In [ ]:
bin_counts = df_stratified_training["moisture_bin"].value_counts()
bin_weights = 1 / bin_counts
df_stratified_training["sample_weight"] = df_stratified_training["moisture_bin"].map(bin_weights)
df_stratified_training["sample_weight"] = df_stratified_training["sample_weight"].astype("float64")
weights = df_stratified_training["sample_weight"]

In [ ]:
bin_counts = bin_counts.sort_index()

plt.figure(figsize=(8, 6), dpi=300)
bars = plt.bar(bin_counts.index.astype(str), bin_counts.values)

for i, bar in enumerate(bars):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        10,
        str(bin_counts.values[i]),
        ha="center",
        va="bottom",
        fontsize=16,
        rotation=90,
        color="black",
    )

plt.title("Measurements counts in soil moisture bins", fontsize=16)
plt.xlabel("Soil moisture bin (%)", fontsize=16)
plt.ylabel("Measurements count", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="y", linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig("dataset_distribution.png")

plt.show()

In [ ]:
for dataset in [df_stratified_training, df_stratified_validation]:
    dataset.drop(columns=["moisture_bin", "sample_weight"], inplace=True, errors="ignore")

In [ ]:
X_training, y_training = data_analysis.split_data(df_stratified_training, TARGET)
X_validation, y_validation = data_analysis.split_data(df_stratified_validation, TARGET)

## Train model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
if TUNE_HYPERPARAMETERS:
    from sklearn.model_selection import GridSearchCV
    
    rf = RandomForestRegressor(random_state=SPLIT_RANDOM_STATE)

    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=HYPERPARAMETERS_GRID,
        cv=5,
        scoring="r2", # "r2" or "neg_mean_squared_error"
        n_jobs=-1,
        verbose=2
    )

    grid_search.fit(X_training, y_training, sample_weight=weights)

    print("Best parameters:", grid_search.best_params_)

    rf_model = grid_search.best_estimator_
else:
    print("Skipped parameters tuning")
    
    rf_model = RandomForestRegressor(
        n_estimators=500,
        max_depth=14,
        min_samples_split=2,
        min_samples_leaf=3,
        max_features="sqrt",
        bootstrap=True,
        random_state=SPLIT_RANDOM_STATE,
        n_jobs=-1,
    )
    
    rf_model.fit(X_training, y_training, sample_weight=weights)

In [ ]:
if EXPORT_MODEL:
    import joblib
    
    joblib.dump(rf_model, f"random_forest_{SELECTED_TARGET}.pkl")

## Analyze model performance

In [ ]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd

In [ ]:
rf_pred = rf_model.predict(X_validation)

rmse = root_mean_squared_error(y_validation, rf_pred)
r_squared = rf_model.score(X_validation, y_validation)

print("Performance for unknown data:")
print(f"Root mean Squared Error: {rmse:.2f}")
print(f"R-squared value: {r_squared:.2f}")

In [ ]:
rf_pred_training = rf_model.predict(X_training)

rmse_training = root_mean_squared_error(y_training, rf_pred_training)
r_squared_training = rf_model.score(X_training, y_training)

print("Performance for known data:")
print(f"Mean Squared Error: {rmse_training:.2f}")
print(f"R-squared value: {r_squared_training:.2f}")

In [ ]:
from typing import Tuple

def rounded_range(data: pd.Series, resolution: int = 10) -> Tuple[int, int]:
    bottom = round(data.min() / resolution - 0.5) * resolution
    top = round(data.max() / resolution + 0.5) * resolution

    return (bottom, top)

In [ ]:
_, top_y = rounded_range(y_validation, resolution=10)
_, top_rf_pred = rounded_range(rf_pred, resolution=10)

axis_min = 0
axis_max = max(top_y, top_rf_pred)

plt.figure(figsize=(10, 8), dpi=300)

sns.scatterplot(x=y_training, y=rf_model.predict(X_training), color="red", label="Predictions on train dataset")
sns.scatterplot(x=y_validation, y=rf_pred, color="blue", label="Predictions on validation dataset")
p = sns.regplot(x=y_validation, y=rf_pred, scatter=False, color="blue", label="Regression line (validation)")
slope, intercept, r, p, sterr = scipy.stats.linregress(x=p.get_lines()[0].get_xdata(), y=p.get_lines()[0].get_ydata())

plt.plot([axis_min, axis_max], [axis_min, axis_max], "r--", label="Perfect prediction")

plt.xlabel("Measured soil moisture (%)", fontsize=16)
plt.ylabel("Predicted soil moisture (%)", fontsize=16)
plt.title(f"Random forest regression prediction of soil moisture\nRMSE: {rmse:.2f}, R²: {r_squared:.2f}, y = {slope:.3f} x + {intercept:.3f}", fontsize=16, fontweight="bold")
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)
plt.grid(True)
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
plt.gca().set_aspect("equal", adjustable="box")
plt.tight_layout()

plt.savefig(f"prediction_test_rf_{TARGET}_params_{PARAMETERS_SET}.png")
plt.show()

In [ ]:
coeffs = pd.Series(rf_model.feature_importances_, index=X_training.columns)
print(coeffs.sort_values(ascending=False))

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_model, X_training, y_training, cv=5, scoring="neg_root_mean_squared_error")
print(f"Cross-validated RMSE: {-scores.mean():.2f} +/- {scores.std():.2f}")

In [ ]:
train_errors = []
test_errors = []

for estimators in range(100, 1300, 200):
    model = RandomForestRegressor(
        n_estimators=estimators,
        max_depth=14,
        min_samples_split=2,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=True,
        random_state=SPLIT_RANDOM_STATE,
        n_jobs=-1,
    )
    model.fit(X_training, y_training, sample_weight=weights)
    
    train_pred = model.predict(X_training)
    test_pred = model.predict(X_validation)
    
    train_rmse = root_mean_squared_error(y_training, train_pred)
    test_rmse = root_mean_squared_error(y_validation, test_pred)
    
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)
plt.plot(range(100, 1300, 200), train_errors, label="Train data")
plt.plot(range(100, 1300, 200), test_errors, label="Test data")
plt.xlabel("n_estimators")
plt.ylabel("RMSE")
plt.title("RMSE n_estimators")
plt.legend()
plt.show()

In [ ]:
train_errors = []
test_errors = []

for depth in range(2, 20, 2):
    model = RandomForestRegressor(
        n_estimators=500,
        max_depth=depth,
        min_samples_split=2,
        min_samples_leaf=3,
        max_features="sqrt",
        bootstrap=True,
        random_state=SPLIT_RANDOM_STATE
    )
    model.fit(X_training, y_training, sample_weight=weights)
    
    train_pred = model.predict(X_training)
    test_pred = model.predict(X_validation)
    
    train_rmse = root_mean_squared_error(y_training, train_pred)
    test_rmse = root_mean_squared_error(y_validation, test_pred)
    
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)

plt.plot(range(2, 20, 2), train_errors, label="Train data")
plt.plot(range(2, 20, 2), test_errors, label="Test data")
plt.xlabel("max_depth")
plt.ylabel("RMSE")
plt.title("RMSE max_depth")
plt.legend()
plt.show()

## Generate output file

In [ ]:
text_file_name = f"rf_parameters_set_{PARAMETERS_SET}.txt"

with open(text_file_name, "w") as f:
    if TUNE_HYPERPARAMETERS:
        f.write("Best parameters:\n")
        for parameter in grid_search.best_params_:
            f.write(f"{parameter}: {grid_search.best_params_[parameter]}\n\n")
    f.write("Performance for unknown data:\n")
    f.write(f"Root Mean Squared Error: {rmse:.2f}\n")
    f.write(f"R-squared value: {r_squared:.2f}\n")
    f.write("\nPerformance for known data:\n")
    f.write(f"Root Mean Squared Error: {rmse_training:.2f}\n")
    f.write(f"R-squared value: {r_squared_training:.2f}\n")
    f.write("\nFeature importances:\n")
    f.write(coeffs.sort_values(ascending=False).to_string())
    f.write(f"\nCross-validated RMSE: {-scores.mean():.2f} +/- {scores.std():.2f}\n")